Stability testing Mk1

In [47]:
import numpy as np
import math
from ChargedDisk.BaseFuncs import Mult_pass as MP
import matplotlib.pyplot as plt
disk_radius = 100
angles = np.linspace(0, 2*np.pi, 100)
x = disk_radius*np.cos(angles)
y = disk_radius*np.sin(angles)

def test_stability(final_configuration, disk_radius=100, Probability = 0.9, temperature_reduction = 0.95, charge_movements = 100, runs = 3, delta = 3, decimals = 8,delta_perturbation = 30):
    test_coords = np.tile(final_configuration, (runs, 1, 1))# generates replicas of the system
    random_move = np.random.uniform(-1,1, size = (runs,test_coords.shape[-2],test_coords.shape[-1])) # generates a random direction to move 
    random_move = random_move/np.linalg.norm(random_move,axis=-1)[...,np.newaxis] # normalises so the distance moved is 1 unit
    temp_coords = test_coords+np.random.rand(random_move.shape[0],random_move.shape[1],1)*delta_perturbation*random_move # shifts the original configurations in the random directions with distance scaled from 0 -> delta_perturbation
    
    # makes sure the charges don't go outside the radius of the disk
    index_run, index_charge = np.where(np.linalg.norm(temp_coords, axis = -1) > 100)
    while index_run.shape[0]>0:
        random_move[index_run,index_charge] = np.random.uniform(-1,1, size = random_move[index_run,index_charge].shape)
        random_move[index_run,index_charge] = random_move[index_run,index_charge]/np.linalg.norm(random_move[index_run,index_charge],axis=-1)[...,np.newaxis]
        temp_coords[index_run,index_charge] = test_coords[index_run,index_charge]+np.random.rand()*delta_perturbation*random_move[index_run,index_charge]
        index_run, index_charge = np.where(np.linalg.norm(temp_coords, axis = -1) > 100)
    test_coords = temp_coords.copy()
    
    # generates the initial temperature in the original way by randomly placing charges and getting the standard deviation
    accuracy = 4
    sample_size = np.power(10,accuracy)
    charges = MP.gen_coords(sample_size, disk_radius, final_configuration.shape[0])
    energies = np.squeeze(np.round(MP.calc_energy(charges, charge=1), accuracy))
    # # this piece of code would not be stable for a high number of charges and is not devloped upon further
    # split_arr = np.split(charges,int(sample_size/np.power(10,5))) # the generated array has to be split otherwise the array size when calculating the energy becomes too large for the RAM to handle
    # energies = np.squeeze(np.round(MP.calc_energy(split_arr[0], charge=1), accuracy))
    # for arr in range(1,len(split_arr)): # the for loop splits up the array and deals with the 'too much ram issue'
    #     print('energy arr {} / {} complete'.format(arr,len(split_arr)))
    #     energies = np.concatenate((energies,np.squeeze(np.round(MP.calc_energy(split_arr[arr], charge=1), accuracy)))) # calculate the energies to the degree of accuracy
    sigma = np.round(np.std(energies), accuracy) # calculate the standard deviation
    print('sigma', sigma)

    # calculates the temperature
    temp = - 3 / math.log(Probability)
    temp *= sigma # calculate the initial temperature from the standard deviation 
    print('initial temperature', temp)

    temp_coords = MP.Choose_Charge(test_coords, delta, disk_radius) # creates a temporary array of coordinates with the moved charge
    updated_coords, energy = MP.Accept_change(temp_coords, test_coords, temp) # updates the coordinates and energy after checking if the conditions are satisfied
    energy_lst = energy[:,np.newaxis] # creates an array of energy values for each simulation
    iters = 1 # sets the iterations to 1 as one pass has been made
    final_coords = np.empty(updated_coords.shape) # creates a template array in which to place the final coordinates
    completed_sims_arr = np.zeros(runs) # creates an array to check when all the simulations have finished
    prev_sum = 0
    while True:
        # each loop of the while statement here is a reduction of temperature
        # runs until a condition is met
        for j in range(charge_movements):
            # each loop here moves a charge.
            temp_coords = MP.Choose_Charge(updated_coords, delta, disk_radius) # create a temporary array of the new coordinates
            updated_coords, energy = MP.Accept_change(
                temp_coords, updated_coords, temp) # decides to accept or reject the new coordinates
        energy_lst = np.append(energy_lst, energy[:,np.newaxis], axis = -1) # appends the energy after each iteration to the energies array
        
        if iters % 100 == 0: # checks every 100 iterations, arbitrary choice but gives good results
            indexes = np.where(\
                np.round(energy_lst[:,iters], decimals = decimals)\
                      == np.round(energy_lst[:,iters-100]\
                        , decimals = decimals))[0] # finds the indexes where the energy has not changed to 'decimals' decimal places
            completed_sims_arr[indexes] = 1 # updates the array to say that a simulation has finished
            final_coords[indexes] = updated_coords[indexes] # places the run simulation coordinates into a final array
            if np.sum(completed_sims_arr) == runs:
                # if the sum of the completed sims array is equal to the number of simulations running
                # all the simulations have run, so the program stops
                break
        if iters % 500 == 0:
            if np.sum(completed_sims_arr) > prev_sum:
                # used to update me on the progress of the simulations
                print('{} out of {} simulations completed.'.format(np.sum(completed_sims_arr), runs))
                prev_sum = np.sum(completed_sims_arr)
        iters += 1 # increase the number of iterations
        temp = temp * temperature_reduction # reduce the temperature, the temperature reduction has already been calculated in the first simulations

    return final_coords, energy_lst, iters, test_coords # return the final state coordinates, the list of energies and the number of iterations


# running the models and saving the data to a file
charge_lst = np.arange(17,25,1)
charge_lst = np.append(charge_lst, 32)
runs = 40
# for charge_num in charge_lst:
charge_num = 32
with np.load('models/simulation {} charges.npz'.format(charge_num)) as data:
    final_coords, energy_lst, _, test_coords = test_stability(data['coords'], disk_radius= 100, temperature_reduction=data['temperature reduction'], runs = runs)

    data_dictionary = {}
    data_dictionary['final coords'] = final_coords
    data_dictionary['perturbation coords'] = test_coords
    data_dictionary['final energies'] = energy_lst

    np.savez('StabilityTestedModelsMk3/{} charges.npz'.format(charge_num),**data_dictionary)

sigma 2.6278
initial temperature 74.82309621189115
1.0 out of 40 simulations completed.
4.0 out of 40 simulations completed.
17.0 out of 40 simulations completed.
28.0 out of 40 simulations completed.
36.0 out of 40 simulations completed.


Stability testing Mk2


In [45]:
import numpy as np
import math
from ChargedDisk.BaseFuncs import Mult_pass as MP
import matplotlib.pyplot as plt
disk_radius = 100
angles = np.linspace(0, 2*np.pi, 100)
x = disk_radius*np.cos(angles)
y = disk_radius*np.sin(angles)

def test_stability2(final_configuration, disk_radius=100, Probability = 0.9, temperature_reduction = 0.95, charge_movements = 100, runs = 3, delta = 3, decimals = 8,delta_perturbation = 30):
    test_coords = np.tile(final_configuration, (runs, 1, 1)) # generates the number of replicas of the system
    random_move = np.random.randint(final_configuration.shape[0], size = runs) # chooses randomly a charge from each replica of the system
    angles = np.random.uniform(0,2*np.pi,size = runs)
    new_coord = np.random.uniform(0,disk_radius,size = runs)[...,np.newaxis]* np.stack((np.cos(angles),np.sin(angles)), axis = -1) # places the charge radnomly at a new position on the disk
    test_coords[np.arange(runs),random_move] = new_coord 
    
    # generates the standard deviation from initialisations of the system to get the average energy
    accuracy = 4
    sample_size = np.power(10,accuracy)
    charges = MP.gen_coords(sample_size, disk_radius, final_configuration.shape[0])
    energies = np.squeeze(np.round(MP.calc_energy(charges, charge=1), accuracy))
    # # this piece of code would not be stable for a high number of charges and is not devloped upon further
    # split_arr = np.split(charges,int(sample_size/np.power(10,5))) # the generated array has to be split otherwise the array size when calculating the energy becomes too large for the RAM to handle
    # energies = np.squeeze(np.round(MP.calc_energy(split_arr[0], charge=1), accuracy))
    # for arr in range(1,len(split_arr)): # the for loop splits up the array and deals with the 'too much ram issue'
    #     print('energy arr {} / {} complete'.format(arr,len(split_arr)))
    #     energies = np.concatenate((energies,np.squeeze(np.round(MP.calc_energy(split_arr[arr], charge=1), accuracy)))) # calculate the energies to the degree of accuracy
    sigma = np.round(np.std(energies), accuracy) # calculate the standard deviation
    print('sigma', sigma)

    temp = - 3 / math.log(Probability)
    temp *= sigma # calculate the initial temperature from the standard deviation 
    print('initial temperature', temp)

    temp_coords = MP.Choose_Charge(test_coords, delta, disk_radius) # creates a temporary array of coordinates with the moved charge
    updated_coords, energy = MP.Accept_change(temp_coords, test_coords, temp) # updates the coordinates and energy after checking if the conditions are satisfied
    energy_lst = energy[:,np.newaxis] # creates an array of energy values for each simulation
    iters = 1 # sets the iterations to 1 as one pass has been made
    final_coords = np.empty(updated_coords.shape) # creates a template array in which to place the final coordinates
    completed_sims_arr = np.zeros(runs) # creates an array to check when all the simulations have finished
    prev_sum = 0
    while True:
        # each loop of the while statement here is a reduction of temperature
        # runs until a condition is met
        for j in range(charge_movements):
            # each loop here moves a charge.
            temp_coords = MP.Choose_Charge(updated_coords, delta, disk_radius) # create a temporary array of the new coordinates
            updated_coords, energy = MP.Accept_change(
                temp_coords, updated_coords, temp) # decides to accept or reject the new coordinates
        energy_lst = np.append(energy_lst, energy[:,np.newaxis], axis = -1) # appends the energy after each iteration to the energies array
        
        if iters % 100 == 0: # checks every 100 iterations, arbitrary choice but gives good results
            indexes = np.where(\
                np.round(energy_lst[:,iters], decimals = decimals)\
                      == np.round(energy_lst[:,iters-100]\
                        , decimals = decimals))[0] # finds the indexes where the energy has not changed to 'decimals' decimal places
            completed_sims_arr[indexes] = 1 # updates the array to say that a simulation has finished
            final_coords[indexes] = updated_coords[indexes] # places the run simulation coordinates into a final array
            if np.sum(completed_sims_arr) == runs:
                # if the sum of the completed sims array is equal to the number of simulations running
                # all the simulations have run, so the program stops
                break
        if iters % 500 == 0:
            if np.sum(completed_sims_arr) > prev_sum:
                # used to update me on the progress of the simulations
                print('{} out of {} simulations completed.'.format(np.sum(completed_sims_arr), runs))
                prev_sum = np.sum(completed_sims_arr)
        iters += 1 # increase the number of iterations
        temp = temp * temperature_reduction # reduce the temperature

    return final_coords, energy_lst, iters, test_coords # return the final state coordinates, the list of energies and the number of iterations


# running the model and saving data
charge_lst = np.arange(17,25,1)
charge_lst = np.append(charge_lst, 32)
runs = 40
# for charge_num in charge_lst:
charge_num = 32
with np.load('models/simulation {} charges.npz'.format(charge_num)) as data:
    final_coords, energy_lst, _, test_coords = test_stability2(data['coords'], disk_radius= 100, temperature_reduction=data['temperature reduction'], runs = runs)

    data_dictionary = {}
    data_dictionary['final coords'] = final_coords
    data_dictionary['perturbation coords'] = test_coords
    data_dictionary['final energies'] = energy_lst

    np.savez('StabilityTestedModels/{} charges.npz'.format(charge_num),**data_dictionary)

(40, 2)
sigma 2.4353
initial temperature 69.34191574884638
1.0 out of 40 simulations completed.
3.0 out of 40 simulations completed.
22.0 out of 40 simulations completed.
32.0 out of 40 simulations completed.
36.0 out of 40 simulations completed.
38.0 out of 40 simulations completed.


Stability testing Mk3

In [49]:
import numpy as np
import math
from ChargedDisk.BaseFuncs import Mult_pass as MP
import matplotlib.pyplot as plt
disk_radius = 100
angles = np.linspace(0, 2*np.pi, 100)
x = disk_radius*np.cos(angles)
y = disk_radius*np.sin(angles)

# The only change from Mk1 is that the standard deviation of the energies is taken from the perturbed system rather than an fresh system.
def test_stability(final_configuration, disk_radius=100, Probability = 0.9, temperature_reduction = 0.95, charge_movements = 100, runs = 3, delta = 3, decimals = 8,delta_perturbation = 30):
    test_coords = np.tile(final_configuration, (runs, 1, 1))
    random_move = np.random.uniform(-1,1, size = (runs,test_coords.shape[-2],test_coords.shape[-1]))
    random_move = random_move/np.linalg.norm(random_move,axis=-1)[...,np.newaxis]
    temp_coords = test_coords+np.random.rand(random_move.shape[0],random_move.shape[1],1)*delta_perturbation*random_move
    index_run, index_charge = np.where(np.linalg.norm(temp_coords, axis = -1) > 100)
    while index_run.shape[0]>0:
        random_move[index_run,index_charge] = np.random.uniform(-1,1, size = random_move[index_run,index_charge].shape)
        random_move[index_run,index_charge] = random_move[index_run,index_charge]/np.linalg.norm(random_move[index_run,index_charge],axis=-1)[...,np.newaxis]
        temp_coords[index_run,index_charge] = test_coords[index_run,index_charge]+np.random.rand()*delta_perturbation*random_move[index_run,index_charge]
        index_run, index_charge = np.where(np.linalg.norm(temp_coords, axis = -1) > 100)
    test_coords = temp_coords.copy()
    
    # finds the standard deviation of energies of all the perturbed systems
    energies = np.squeeze(np.round(MP.calc_energy(test_coords, charge=1)))
    # # this piece of code would not be stable for a high number of charges and is not devloped upon further
    # split_arr = np.split(charges,int(sample_size/np.power(10,5))) # the generated array has to be split otherwise the array size when calculating the energy becomes too large for the RAM to handle
    # energies = np.squeeze(np.round(MP.calc_energy(split_arr[0], charge=1), accuracy))
    # for arr in range(1,len(split_arr)): # the for loop splits up the array and deals with the 'too much ram issue'
    #     print('energy arr {} / {} complete'.format(arr,len(split_arr)))
    #     energies = np.concatenate((energies,np.squeeze(np.round(MP.calc_energy(split_arr[arr], charge=1), accuracy)))) # calculate the energies to the degree of accuracy
    sigma = np.std(energies) # calculate the standard deviation
    print('sigma', sigma)

    temp = - 3 / math.log(Probability)
    temp *= sigma # calculate the initial temperature from the standard deviation 
    print('initial temperature', temp)

    temp_coords = MP.Choose_Charge(test_coords, delta, disk_radius) # creates a temporary array of coordinates with the moved charge
    updated_coords, energy = MP.Accept_change(temp_coords, test_coords, temp) # updates the coordinates and energy after checking if the conditions are satisfied
    energy_lst = energy[:,np.newaxis] # creates an array of energy values for each simulation
    iters = 1 # sets the iterations to 1 as one pass has been made
    final_coords = np.empty(updated_coords.shape) # creates a template array in which to place the final coordinates
    completed_sims_arr = np.zeros(runs) # creates an array to check when all the simulations have finished
    prev_sum = 0
    while True:
        # each loop of the while statement here is a reduction of temperature
        # runs until a condition is met
        for j in range(charge_movements):
            # each loop here moves a charge.
            temp_coords = MP.Choose_Charge(updated_coords, delta, disk_radius) # create a temporary array of the new coordinates
            updated_coords, energy = MP.Accept_change(
                temp_coords, updated_coords, temp) # decides to accept or reject the new coordinates
        energy_lst = np.append(energy_lst, energy[:,np.newaxis], axis = -1) # appends the energy after each iteration to the energies array
        
        if iters % 100 == 0: # checks every 100 iterations, arbitrary choice but gives good results
            indexes = np.where(\
                np.round(energy_lst[:,iters], decimals = decimals)\
                      == np.round(energy_lst[:,iters-100]\
                        , decimals = decimals))[0] # finds the indexes where the energy has not changed to 'decimals' decimal places
            completed_sims_arr[indexes] = 1 # updates the array to say that a simulation has finished
            final_coords[indexes] = updated_coords[indexes] # places the run simulation coordinates into a final array
            if np.sum(completed_sims_arr) == runs:
                # if the sum of the completed sims array is equal to the number of simulations running
                # all the simulations have run, so the program stops
                break
        if iters % 500 == 0:
            if np.sum(completed_sims_arr) > prev_sum:
                # used to update me on the progress of the simulations
                print('{} out of {} simulations completed.'.format(np.sum(completed_sims_arr), runs))
                prev_sum = np.sum(completed_sims_arr)
        iters += 1 # increase the number of iterations
        temp = temp * temperature_reduction # reduce the temperature

    return final_coords, energy_lst, iters, test_coords # return the final state coordinates, the list of energies and the number of iterations


# running and saving the model data
charge_lst = np.arange(17,25,1)
charge_lst = np.append(charge_lst, 32)
runs = 40

# for charge_num in charge_lst:
charge_num = 32
with np.load('models/simulation {} charges.npz'.format(charge_num)) as data:
    final_coords, energy_lst, _, test_coords = test_stability(data['coords'], disk_radius= 100, temperature_reduction=data['temperature reduction'], runs = runs)
    data_dictionary = {}
    data_dictionary['final coords'] = final_coords
    data_dictionary['perturbation coords'] = test_coords
    data_dictionary['final energies'] = energy_lst

    np.savez('StabilityTestedModelsMk4/{} charges.npz'.format(charge_num),**data_dictionary)

sigma 0.5425633603552676
initial temperature 15.448767224640063
1.0 out of 40 simulations completed.
11.0 out of 40 simulations completed.
25.0 out of 40 simulations completed.
34.0 out of 40 simulations completed.
38.0 out of 40 simulations completed.
